In [1]:
from skimage import io, color
from skimage.morphology import binary_closing, binary_opening
from skimage.morphology import disk
import matplotlib.pyplot as plt
import numpy as np
from skimage import measure
from skimage.color import label2rgb
import pydicom as dicom
from scipy.stats import norm
from scipy.spatial import distance

def show_comparison(original, modified, modified_name):
    fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(8, 4), sharex=True,
                                   sharey=True)
    ax1.imshow(original, cmap="gray", vmin=-200, vmax=500)
    ax1.set_title('Original')
    ax1.axis('off')
    ax2.imshow(modified)
    ax2.set_title(modified_name)
    ax2.axis('off')
    io.show()

%matplotlib qt

Exercise 1

In [ ]:
from skimage import io
import pydicom as dicom
import matplotlib.pyplot as plt

# Read the CT scan image
in_dir = "data/"
ct = dicom.dcmread(in_dir + 'Training.dcm')
img = ct.pixel_array

# Visualize with good contrast for spleen
plt.figure(figsize=(6,6))
io.imshow(img, vmin=0, vmax=150, cmap='gray')
io.show()


Exercise 2

In [ ]:
import numpy as np
from skimage import io
import pydicom as dicom

# Load DICOM image
in_dir = "data/"
ct = dicom.dcmread(in_dir + 'Training.dcm')
img = ct.pixel_array

# Load spleen mask and convert to boolean
spleen_roi = io.imread(in_dir + 'SpleenROI.png')
spleen_mask = spleen_roi > 0

# Extract HU values from spleen region
spleen_values = img[spleen_mask]

# Compute statistics
spleen_mean = np.mean(spleen_values)
spleen_std = np.std(spleen_values)

print(f"Spleen mean HU: {spleen_mean:.2f}")
print(f"Spleen standard deviation: {spleen_std:.2f}")


Exercise 3

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from skimage import io
import pydicom as dicom

# Load CT image and spleen mask
in_dir = "data/"
ct = dicom.dcmread(in_dir + 'Training.dcm')
img = ct.pixel_array
spleen_mask = io.imread(in_dir + 'SpleenROI.png') > 0

# Extract spleen HU values
spleen_values = img[spleen_mask]

# Compute mean and std
mu_spleen = np.mean(spleen_values)
std_spleen = np.std(spleen_values)

# Plot histogram of spleen values
n, bins, patches = plt.hist(spleen_values, bins=60, density=True, alpha=0.6, color='gray', label='Spleen pixel values')

# Plot fitted Gaussian
pdf_spleen = norm.pdf(bins, mu_spleen, std_spleen)
plt.plot(bins, pdf_spleen, 'r-', label='Fitted Gaussian')

# Add labels and legend
plt.xlabel('Hounsfield Unit')
plt.ylabel('Density')
plt.title('Spleen HU Value Distribution')
plt.legend()
plt.show()


Exercise 4

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from skimage import io
import pydicom as dicom

# Load CT image
in_dir = "data/"
ct = dicom.dcmread(in_dir + 'Training.dcm')
img = ct.pixel_array

# Load all tissue masks
masks = {
    "spleen": io.imread(in_dir + 'SpleenROI.png') > 0,
    "bone": io.imread(in_dir + 'BoneROI.png') > 0,
    "fat": io.imread(in_dir + 'FatROI.png') > 0,
    "kidney": io.imread(in_dir + 'KidneyROI.png') > 0,
    "liver": io.imread(in_dir + 'LiverROI.png') > 0
}

# Compute HU values, mean, std for each tissue
tissue_stats = {}
for tissue, mask in masks.items():
    values = img[mask]
    mu = np.mean(values)
    std = np.std(values)
    tissue_stats[tissue] = {
        "values": values,
        "mean": mu,
        "std": std
    }

# Plot histograms
plt.figure(figsize=(12, 6))
for tissue, stats in tissue_stats.items():
    plt.hist(stats["values"], bins=60, alpha=0.4, density=True, label=tissue)
plt.xlabel("Hounsfield Unit")
plt.ylabel("Density")
plt.title("Histograms of Tissues")
plt.legend()
plt.show()

# Plot fitted Gaussians
plt.figure(figsize=(12, 6))
min_hu = -200
max_hu = 1000
hu_range = np.arange(min_hu, max_hu, 1.0)

colors = {
    "spleen": 'r--',
    "bone": 'g-',
    "fat": 'b--',
    "kidney": 'm-',
    "liver": 'orange'
}

for tissue, stats in tissue_stats.items():
    pdf = norm.pdf(hu_range, stats["mean"], stats["std"])
    plt.plot(hu_range, pdf, colors.get(tissue, 'k-'), label=tissue)

plt.xlabel("Hounsfield Unit")
plt.ylabel("PDF")
plt.title("Fitted Gaussians for Tissues")
plt.legend()
plt.show()


Exercise 5

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

# Define HU plotting range
min_hu = -200
max_hu = 1000
hu_range = np.arange(min_hu, max_hu, 1.0)

# Use same `tissue_stats` from Exercise 4
colors = {
    "spleen": 'r--',
    "bone": 'g-',
    "fat": 'b--',
    "kidney": 'm-',
    "liver": 'orange'
}

plt.figure(figsize=(12, 6))
for tissue, stats in tissue_stats.items():
    pdf = norm.pdf(hu_range, stats["mean"], stats["std"])
    plt.plot(hu_range, pdf, colors.get(tissue, 'k-'), label=tissue)

plt.xlabel("Hounsfield Unit")
plt.ylabel("PDF")
plt.title("Fitted Gaussian Distributions for Tissues")
plt.legend()
plt.grid(True)
plt.show()


Exercise 6

In [ ]:
classes = ["background", "fat", "bone", "soft tissue"]

classes = ["fat", "bone", "soft tissue"]



Exercise 7

In [ ]:
# Step 1: Load masks
fat_mask = io.imread(in_dir + 'FatROI.png') > 0
bone_mask = io.imread(in_dir + 'BoneROI.png') > 0
spleen_mask = io.imread(in_dir + 'SpleenROI.png') > 0
liver_mask = io.imread(in_dir + 'LiverROI.png') > 0
kidney_mask = io.imread(in_dir + 'KidneyROI.png') > 0

# Step 2: Extract values
fat_values = img[fat_mask]
bone_values = img[bone_mask]
spleen_values = img[spleen_mask]
liver_values = img[liver_mask]
kidney_values = img[kidney_mask]

# Step 3: Compute mean HU for each class
mean_fat = np.mean(fat_values)
mean_bone = np.mean(bone_values)
mean_soft = np.mean(np.concatenate((spleen_values, liver_values, kidney_values)))

# Step 4: Compute thresholds
t_background = -200
t_fat_soft = (mean_fat + mean_soft) / 2
t_soft_bone = (mean_soft + mean_bone) / 2

print(f"Fat-soft tissue threshold: {t_fat_soft:.2f}")
print(f"Soft tissue-bone threshold: {t_soft_bone:.2f}")


Exercise 8

In [ ]:
from skimage.color import label2rgb
import matplotlib.pyplot as plt

# Step 1: Apply thresholds
fat_img = (img > t_background) & (img <= t_fat_soft)
soft_img = (img > t_fat_soft) & (img <= t_soft_bone)
bone_img = img > t_soft_bone

# Step 2: Combine into a labeled image
label_img = np.zeros_like(img, dtype=np.uint8)
label_img[fat_img] = 1
label_img[soft_img] = 2
label_img[bone_img] = 3

# Optional: for visual contrast in label2rgb (not needed but cleaner)
image_label_overlay = label2rgb(label_img, image=img, bg_label=0, alpha=0.5)

# Step 3: Show original + overlay
def show_comparison(original, overlay, title=''):
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.imshow(original, cmap='gray')
    plt.title('Original CT Slice')
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(overlay)
    plt.title(title)
    plt.axis('off')

    plt.tight_layout()
    plt.show()

# Call the visualization function
show_comparison(img, image_label_overlay, 'Classification result')


Exercise 9

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

# Step 1: Define HU range
min_hu = -400
max_hu = 1000
hu_range = np.arange(min_hu, max_hu, 1.0)

# Step 2: Compute fitted Gaussians (reusing tissue_stats from earlier)
pdf_fat = norm.pdf(hu_range, tissue_stats["fat"]["mean"], tissue_stats["fat"]["std"])
pdf_soft = norm.pdf(hu_range, tissue_stats["spleen"]["mean"], tissue_stats["spleen"]["std"])  # or use mean of all soft tissues
pdf_liver = norm.pdf(hu_range, tissue_stats["liver"]["mean"], tissue_stats["liver"]["std"])
pdf_kidney = norm.pdf(hu_range, tissue_stats["kidney"]["mean"], tissue_stats["kidney"]["std"])
pdf_bone = norm.pdf(hu_range, tissue_stats["bone"]["mean"], tissue_stats["bone"]["std"])

# Optionally average spleen, liver, kidney into one soft tissue Gaussian
mu_soft = np.mean([tissue_stats[t]["mean"] for t in ["spleen", "liver", "kidney"]])
std_soft = np.mean([tissue_stats[t]["std"] for t in ["spleen", "liver", "kidney"]])
pdf_soft_combined = norm.pdf(hu_range, mu_soft, std_soft)

# Step 3: Plot
plt.figure(figsize=(12, 6))
plt.plot(hu_range, pdf_fat, 'b', label='Fat')
plt.plot(hu_range, pdf_soft_combined, 'm', label='Soft Tissue (combined)')
plt.plot(hu_range, pdf_bone, 'g', label='Bone')
plt.axvline(x=-200, color='k', linestyle='--', label='Background threshold')
plt.xlabel("Hounsfield Unit")
plt.ylabel("PDF")
plt.title("Fitted Gaussians for Parametric Classification")
plt.legend()
plt.grid(True)
plt.show()


Execise 10

In [ ]:
# Step 1: Create lookup table for HU values in [-200, 2000] range
hu_vals = np.arange(-200, 2000)  # You can adapt this range if needed

# Gaussian parameters (using combined soft tissue stats)
mu_fat = tissue_stats["fat"]["mean"]
std_fat = tissue_stats["fat"]["std"]

mu_bone = tissue_stats["bone"]["mean"]
std_bone = tissue_stats["bone"]["std"]

mu_soft = np.mean([tissue_stats[t]["mean"] for t in ["spleen", "liver", "kidney"]])
std_soft = np.mean([tissue_stats[t]["std"] for t in ["spleen", "liver", "kidney"]])

# Step 2: Compute norm.pdf for each class over HU values
p_fat = norm.pdf(hu_vals, mu_fat, std_fat)
p_soft = norm.pdf(hu_vals, mu_soft, std_soft)
p_bone = norm.pdf(hu_vals, mu_bone, std_bone)

# Step 3: Determine class by maximum probability
# Background = 0, Fat = 1, Soft Tissue = 2, Bone = 3
lookup_table = np.zeros_like(hu_vals, dtype=np.uint8)
lookup_table[np.argmax(np.vstack([p_fat, p_soft, p_bone]), axis=0) + 1] = 1  # Fat=1
lookup_table[np.argmax(np.vstack([p_fat, p_soft, p_bone]), axis=0) + 1 == 2] = 2  # Soft=2
lookup_table[np.argmax(np.vstack([p_fat, p_soft, p_bone]), axis=0) + 1 == 3] = 3  # Bone=3

# Step 4: Classify the image using the lookup table
classified_img = np.zeros_like(img, dtype=np.uint8)
valid_mask = img >= -200
classified_img[~valid_mask] = 0  # background
clipped_vals = np.clip(img[valid_mask], -200, 1999)
lut_indices = clipped_vals + 200  # shift index to start from 0
classified_img[valid_mask] = lookup_table[lut_indices]

# Step 5: Visualize
from skimage.color import label2rgb
image_label_overlay = label2rgb(classified_img, image=img, bg_label=0, alpha=0.5)

def show_comparison(original, overlay, title=''):
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.imshow(original, cmap='gray')
    plt.title('Original CT Slice')
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(overlay)
    plt.title(title)
    plt.axis('off')

    plt.tight_layout()
    plt.show()

# Display classification result
show_comparison(img, image_label_overlay, 'Parametric Classification Result')


Exercise 11

In [ ]:
from scipy.stats import norm
import numpy as np

# Parameters from previous steps
mu_fat = tissue_stats["fat"]["mean"]
std_fat = tissue_stats["fat"]["std"]

mu_bone = tissue_stats["bone"]["mean"]
std_bone = tissue_stats["bone"]["std"]

mu_soft = np.mean([tissue_stats[t]["mean"] for t in ["spleen", "liver", "kidney"]])
std_soft = np.mean([tissue_stats[t]["std"] for t in ["spleen", "liver", "kidney"]])

# Define HU range to search
hu_range = np.linspace(-200, 1000, 5000)

# Compute PDFs
pdf_fat = norm.pdf(hu_range, mu_fat, std_fat)
pdf_soft = norm.pdf(hu_range, mu_soft, std_soft)
pdf_bone = norm.pdf(hu_range, mu_bone, std_bone)

# Find intersections
diff_fat_soft = np.abs(pdf_fat - pdf_soft)
t_fat_soft_opt = hu_range[np.argmin(diff_fat_soft)]

diff_soft_bone = np.abs(pdf_soft - pdf_bone)
t_soft_bone_opt = hu_range[np.argmin(diff_soft_bone)]


Exercise 12

Exercise 14

In [ ]:
from skimage import measure

# Step 1: Get BLOB features
region_props = measure.regionprops(label_img)

# Step 2: Define thresholds
min_area = 1000
max_area = 5000

min_perimeter = 150
max_perimeter = 600

# Optional: add more features like eccentricity, solidity, etc.

# Step 3: Filter based on multiple criteria
label_img_filter = label_img.copy()
for region in region_props:
    area_ok = min_area <= region.area <= max_area
    peri_ok = min_perimeter <= region.perimeter <= max_perimeter
    # Add more conditions here if needed

    if not (area_ok and peri_ok):
        for coords in region.coords:
            label_img_filter[coords[0], coords[1]] = 0

# Step 4: Create binary mask of valid BLOBs
i_combined = label_img_filter > 0

# Step 5: Visualize
show_comparison(img, i_combined, 'Spleen Filtered by Area + Perimeter')


Exercise 15

In [ ]:
import numpy as np
from scipy.stats import norm
from skimage import morphology, measure

def spleen_finder(img):
    # --- STEP 1: Parameters ---
    t_background = -200
    hu_vals = np.arange(-200, 2000)

    # Gaussian class parameters (these should be trained on your dataset)
    mu_fat = -100   # example values — update if needed
    std_fat = 20

    mu_soft = 50
    std_soft = 15

    mu_bone = 600
    std_bone = 100

    # Step 2: Create lookup table for classification
    p_fat = norm.pdf(hu_vals, mu_fat, std_fat)
    p_soft = norm.pdf(hu_vals, mu_soft, std_soft)
    p_bone = norm.pdf(hu_vals, mu_bone, std_bone)

    class_map = np.argmax(np.vstack([p_fat, p_soft, p_bone]), axis=0) + 1  # 1=fat, 2=soft, 3=bone
    lookup_table = np.zeros_like(hu_vals, dtype=np.uint8)
    lookup_table = class_map

    # Step 3: Apply classification to image
    classified_img = np.zeros_like(img, dtype=np.uint8)
    valid_mask = img >= t_background
    clipped_vals = np.clip(img[valid_mask], -200, 1999)
    lut_indices = clipped_vals + 200  # shift range to index 0
    classified_img[valid_mask] = lookup_table[lut_indices]

    # Step 4: Extract soft tissue class (label 2)
    soft_img = classified_img == 2

    # Step 5: Morphological operations to separate blobs
    soft_clean = morphology.opening(soft_img, morphology.disk(3))
    soft_clean = morphology.closing(soft_clean, morphology.disk(5))

    # Step 6: Label and analyze blobs
    label_img = measure.label(soft_clean)
    region_props = measure.regionprops(label_img)

    # Thresholds based on spleen characteristics
    min_area = 1000
    max_area = 5000
    min_perimeter = 100
    max_perimeter = 500

    label_img_filtered = label_img.copy()
    for region in region_props:
        area_ok = min_area <= region.area <= max_area
        peri_ok = min_perimeter <= region.perimeter <= max_perimeter
        if not (area_ok and peri_ok):
            for coords in region.coords:
                label_img_filtered[coords[0], coords[1]] = 0

    # Step 7: Output binary spleen mask
    spleen_mask = label_img_filtered > 0
    return spleen_mask


Exercise 16

In [ ]:
import pydicom
import matplotlib.pyplot as plt

# File list
validation_files = ["Validation1.dcm", "Validation2.dcm", "Validation3.dcm"]

# Function to run spleen_finder and show results
def test_on_validation_images(in_dir="data/"):
    for fname in validation_files:
        # Load image
        ct = pydicom.dcmread(in_dir + fname)
        img = ct.pixel_array

        # Run spleen detection
        spleen_mask = spleen_finder(img)

        # Show result
        show_comparison(img, spleen_mask, f"Spleen prediction for {fname}")

# Run the tests
test_on_validation_images()


Exercise 17

In [ ]:
from scipy.spatial import distance
from skimage import io
import pydicom

# Validation filenames
validation_images = ["Validation1.dcm", "Validation2.dcm", "Validation3.dcm"]
ground_truth_masks = ["Validation1_spleen.png", "Validation2_spleen.png", "Validation3_spleen.png"]

def compute_dice(pred, truth):
    return 1 - distance.dice(pred.ravel(), truth.ravel())

def evaluate_spleen_detection(in_dir="data/"):
    for i in range(3):
        # Load image and ground truth
        img = pydicom.dcmread(in_dir + validation_images[i]).pixel_array
        gt_img = io.imread(in_dir + ground_truth_masks[i]) > 0

        # Predict spleen mask
        pred_mask = spleen_finder(img)

        # Compute DICE score
        dice = compute_dice(pred_mask, gt_img)
        print(f"DICE score for {validation_images[i]}: {dice:.4f}")
